In [2]:
import sys
sys.path.append('../..')

import requests
import json
import pandas as pd
import os

import sqlite3

In [4]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_OUT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AGRO-CITTCA\API_FINANCIERO'
PARH_XLSX_GRUPOS_UCAF = RUTA_UNIDAD_ONE_DRIVE + '\Ingenio Azucarero Guabira S.A\COMPROMISOS ZAFRA 2025 - Documents\GRUPOS UCAF.xlsx'

In [18]:
def get_token():
    # URL base de la API
    url = "https://guabirasistemas.com:9062/Auth/login"
    # Parámetros de consulta
    params = {
        "pStrUsuario": "USRUCAF",
        "pStrClave": "DC513EA4FBDAA7A14786FFDEBC4EF64E"
    }
    response = requests.get(url, params=params)
    data = json.loads(response.text)
    token = data['evUser']['token']
    return token

In [19]:
TOKEN = get_token()

# /Cml/TipoCreditos

In [20]:
def get_tipo_credito():
    api_url = "https://guabirasistemas.com:9062/Cml/TipoCreditos"
    headers = {
        "Authorization": f"Bearer {TOKEN}"
    }
    response = requests.get(api_url, headers=headers)
    res = response.json()
    tipos_prestamo = res['evTipos']
    df = pd.DataFrame(tipos_prestamo)
    filename = 'TipoCreditos.xlsx'
    out = os.path.join(PATH_OUT, filename)
    df.to_excel(out, index=False)

# /Cml/ObtenerGrupo

In [155]:
# conectar a la base SQLite (creará archivo si no existe)
conn = sqlite3.connect('db_data_agrocittca.db')
cursor = conn.cursor()

# crear tabla
cursor.execute('''
CREATE TABLE IF NOT EXISTS grupo_caneros (
    codgrupo INTEGER,
    codcanero INTEGER,
    nombrecanero TEXT,
    codfinanza INTEGER,
    totaldeuda REAL
)
''')

In [156]:
def get_obtener_grupo(cod_ca):
    api_url = "https://guabirasistemas.com:9062/Cml/ObtenerGrupo"
    headers = {
        "Authorization": f"Bearer {TOKEN}"
    }
    params = {
        "pStrCanero": str(cod_ca)
    }
    response = requests.get(api_url, headers=headers, params=params)
    res = response.json()
    grupos = res['evGrupos']
    df = pd.DataFrame(grupos)
    return df

def seleccionar_datos():
    # conectar a la base SQLite
    conn = sqlite3.connect('db_data_agrocittca.db')
    # leer datos con pandas
    df = pd.read_sql_query('SELECT * FROM grupo_caneros', conn)
    # cerrar conexión
    conn.close()
    return df

def borrar_todos_los_datos():
    # conectar a la base SQLite
    conn = sqlite3.connect('db_data_agrocittca.db')
    cursor = conn.cursor()
    # ejecutar borrado
    cursor.execute('DELETE FROM grupo_caneros')
    # confirmar cambios
    conn.commit()
    conn.close()
    print("Todos los datos de la tabla grupo_caneros han sido borrados.")
    return None
    
def obtener_codigos_ca():
    df = pd.read_excel(PARH_XLSX_GRUPOS_UCAF, sheet_name='GRUPO ECONOMICO')
    df_sin_repetidos = df.drop_duplicates(subset=['GRUPO ECONOMICO'])
    lista_cods = list(set(df_sin_repetidos['CODIGO']))
    return lista_cods

def seleccionar_codcanero_distintos():
    # conectar a la base SQLite
    conn = sqlite3.connect('db_data_agrocittca.db')
    # ejecutar consulta y leer resultados
    df = pd.read_sql_query('SELECT DISTINCT codcanero FROM grupo_caneros', conn)
    # cerrar conexión
    conn.close()
    return list(df['codcanero'])

def obtener_diferencia(lista_a, lista_b):
    # convertir lista_b a set para búsqueda rápida
    set_b = set(lista_b)
    # filtrar los que no están en lista_b
    diferencia = [x for x in lista_a if x not in set_b]
    return diferencia

In [157]:
borrar_todos_los_datos()

Todos los datos de la tabla grupo_caneros han sido borrados.


In [158]:
cods_ucaf = obtener_codigos_ca()
print(f'Se encontraron {len(cods_ucaf)} codigos.')

Se encontraron 857 codigos.


In [159]:
cods_db = seleccionar_codcanero_distintos()
print(f'Se encontraron {len(cods_db)} codigos en la BD')

Se encontraron 0 codigos en la BD


In [160]:
codigos_faltantes = obtener_diferencia(cods_ucaf, cods_db)
print(f'Se encontraron {len(codigos_faltantes)} codigos faltantes en la BD')

Se encontraron 857 codigos faltantes en la BD


In [151]:
for cod in codigos_faltantes:
    intentos = 0
    max_intentos = 3
    while intentos < max_intentos:
        try:
            df_grupo = get_obtener_grupo(cod)

            conn = sqlite3.connect('db_data_agrocittca.db')
            df_grupo.to_sql('grupo_caneros', conn, if_exists='append', index=False)
            conn.close()

            print(f'Se procesó código: {cod}')
            break  # si salió bien, salir del while

        except requests.exceptions.ConnectTimeout:
            intentos += 1
            print(f'Timeout en intento {intentos} para código {cod}')
            time.sleep(5)

        except Exception as e:
            print(f'Error inesperado para código {cod}: {e}')
            break  # para errores que no sean timeout, salir del while

SyntaxError: invalid syntax (3938415019.py, line 1)

In [152]:
df_resultado = seleccionar_datos()
print(f'Se tiene {len(df_resultado)} registros en DB')

Se tiene 3 registros en DB


In [153]:
filename = 'ObtenerGrupo.xlsx'
out = os.path.join(PATH_OUT, filename)
df_resultado.to_excel(out, index=False)

# /Cml/EstadoCuenta

In [162]:
get_obtener_grupo(515)

,codgrupo,codcanero,nombrecanero,codfinanza,totaldeuda
0,262,515,AGUILERA OLGA OLIMPIA RIVERO VDA DE,800078,65298.14
1,262,1215,AGUILERA RIVERO OLGA KARINA,800082,10227.89
2,262,8003,AGUILERA RIVERO SERGIO STEFAN,800083,42901.10
3,262,40004,AGUILERA RIVERO NICOLAS JULIO,800081,19391.80


In [163]:
def get_estado_cuenta(grupo, canero):
    api_url = "https://guabirasistemas.com:9062/Cml/EstadoCuenta"
    headers = {
        "Authorization": f"Bearer {TOKEN}"
    }
    params = {
        "pStrGrupo": grupo,
        "pStrCanero": canero
    }

    try:
        response = requests.get(api_url, headers=headers, params=params, timeout=5)
        response.raise_for_status()  # lanza error si el status != 200

        estado_cuenta = response.text  # porque devuelve text/plain
        return estado_cuenta

    except requests.exceptions.RequestException as e:
        print(f"Error al obtener estado de cuenta: {e}")
        return None

In [176]:
estado = get_estado_cuenta(262, 1215)

In [177]:
data = json.loads(estado)

In [ ]:
df = pd.DataFrame(tipos_prestamo)

In [179]:
data['evData']

[{'nrocontrato': '0000010019105',
  'moneda': 'USD',
  'claseprstamo': 'Préstamo Anticipo Semanal',
  'montocredito': 2574.53,
  'integenerado': 0.0,
  'intercobrado': 0.0,
  'interxcobrar': 0.0,
  'saldopendnte': 0.0,
  'totalpagado': 2574.53,
  'totaldeuda': 0.0,
  'estado': 'PAGADO',
  'diasmora': 0},
 {'nrocontrato': '0000010024156',
  'moneda': 'USD',
  'claseprstamo': 'Préstamo Anticipo Semanal',
  'montocredito': 2625.76,
  'integenerado': 0.0,
  'intercobrado': 0.0,
  'interxcobrar': 0.0,
  'saldopendnte': 0.0,
  'totalpagado': 2625.76,
  'totaldeuda': 0.0,
  'estado': 'PAGADO',
  'diasmora': 0},
 {'nrocontrato': '0000010028091',
  'moneda': 'USD',
  'claseprstamo': 'Préstamo Anticipo Semanal',
  'montocredito': 2993.8,
  'integenerado': 0.0,
  'intercobrado': 0.0,
  'interxcobrar': 0.0,
  'saldopendnte': 0.0,
  'totalpagado': 2993.8,
  'totaldeuda': 0.0,
  'estado': 'PAGADO',
  'diasmora': 0},
 {'nrocontrato': '0000010017084',
  'moneda': 'USD',
  'claseprstamo': 'Préstamo Cañ

In [ ]:
get_tipo_credito()